# Introduction to Orbit Fitting with pyOrbfit

This tutorial introduces the pyOrbfit module, a set of python wrappers for orbit-fitting code written in C by Gary Bernstein. The algorithm is optimized for fitting distant solar system objects and is described in Bernstein and Khushalani, <a href="http://iopscience.iop.org/article/10.1086/316868;jsessionid=34862896C348AFD2F7994AC3870D30FD.c2.iopscience.cld.iop.org">"Orbit Fitting and Uncertainty for Kuiper Belt Objects", AJ 120, 3323 (2000)</a>. This code is an integral part of our TNO-finding toolkit. With it you can perform the following functions:
  * Fit a set of observations (RA, DEC, date, observatory code) to determine the orbital elements and goodness of fit.
  * Predict the position and error ellipse of an object at past or future dates.
 

## Prerequisites:

* A working installation of Python 2.7. The code uses standard packages like numpy and pandas that should be part of most distributions.
* Install the pyephem package if you don't already have it, using your favorite package manager such as pip or easy_install, e.g.   <tt>pip install ephem</tt>

## Obtaining and installing the code:

The code is on github at <a href="https://github.com/dwgerdes/pyOrbfit">https://github.com/dwgerdes/pyOrbfit</a>.

1) Download a copy and place it in its own directory on your machine. I use ~/pyOrbfit. 

2) Move to this directory and compile the code:
   * <tt>python setup.py build</tt>
   
This compiles the C code. You may see some non-fatal warnings about unused variables, etc. but these can be ignored.
   
3) When compilation is finished, look in the <tt>build</tt> subdirectory. There you will see a directory with a machine-dependent name--on my Mac it's lib.macosx-10.8-x86_64-2.7/. (Check the timestamp on the directory to make sure it was just created in this build.) In this directory you'll find a file called

<tt>_pyOrbfit.so</tt>.

This is a shared library containing the compiled code. Copy this file to the main pyOrbfit directory. Do not leave off the leading underscore! 

4) The code needs two auxiliary files, both of which are located in the main directory:

   * <tt>observatories.dat</tt> contains a list of observatory codes and their geographic information. It's a small subset of the full list available from the Minor Planet Center at http://www.minorplanetcenter.net/iau/lists/ObsCodesF.html. Newer observatory codes contain letters, for example "Cerro Tololo / DECam" is W84. pyOrbfit is currently able to handle only integer codes, so we use the more generic "Cerro Tololo Observatory" code 807. Fixing this is an exercise for the reader. (Observatory codes enable us to correct for parallex due to our observing location on earth, or with space-based telescopes. This is unimportant for TNOs but it does matter for main belt asteroids and NEOs. The Minor Planet Center requires all observations to be submitted with a recognized observatory code. We replace the 807s with W84s when we report our observations with DECam.)
   
   * <tt>binEphem.423</tt> This is a compiled version of the JPL DE423 <a href="http://ssd.jpl.nasa.gov/?ephemerides">ephemeris</a>. It contains precision tabulated positions of major (and many minor) solar system bodies over a range of dates +/- several centuries from now. It's used to compute perturbations to TNOs from Neptune and the other giant planets. The compiled version available here *should* work on most machines (we've used it successfully on Mac and Linux), but if you encounter problems on your machine you may need to rebuild it using the tools available on the JPL web site.

5) Define some environmental variables:

  * <tt>ORBIT_OBSERVATORIES</tt> should point to the location of <tt>observatories.dat</tt>
  * <tt>ORBIT_EPHEMERIS</tt> should point to <tt>binEphem.423</tt>
  * Add the main pyOrbfit directory to your PYTHONPATH.
  
  Depending on your machine, you may do this by adding the following lines to your <tt>.bashrc</tt> or <tt>.bash_profile</tt> file:
 
  export PYORBFIT="/home/yourname/pyOrbfit"
  
  export ORBIT_OBSERVATORIES=$\$$PYORBFIT/observatories.dat
  
  export ORBIT_EPHEMERIS=$\$$PYORBFIT/binEphem.423
  
  export PYTHONPATH=$\$$PYTHONPATH:$\$$PYORBFIT
  
 Someone who is better-versed in distutils than I am could probably devise a setup.py file that would make all this happen automatically when the package is installed. 

## Let's Try it Out!

The code is distributed with an example file <tt>2013_TV158.mpc</tt>, which contains observations of the scattered disk object <a href="http://www.minorplanetcenter.net/db_search/show_object?utf8=%E2%9C%93&object_id=2013+TV158">2013 TV158</a> by DECam, Pan-STARRS, and SDSS. They are in <a href="http://www.minorplanetcenter.net/iau/info/OpticalObs.html">Minor Planet Center format</a>.

In [16]:
# import the package:
from Orbit import Orbit
import ephem 
import numpy as np

In [7]:
o = Orbit(file='2013_TV158.mpc')

In [8]:
elements, errs = o.get_elements()
print elements
print errs

{'a': 111.22460233579679, 'lan': 181.0744263335385, 'e': 0.6720959654675543, 'i': 31.142680538860287, 'top': 2459777.3149905493, 'aop': 232.09156280568914}
{'a': 0.0078527132767012304, 'lan': 0.00011397565932540869, 'e': 2.1797880078135241e-05, 'i': 8.6080888604753787e-05, 'top': 0.24862051383178221, 'aop': 0.0015573146902426677}


If the above lines worked, you should see the orbital elements and errors for this object:


{'a': 111.22460233579679, 'lan': 181.0744263335385, 'e': 0.6720959654675543, 'i': 31.142680538860287, 'top': 2459777.3149905493, 'aop': 232.09156280568914}

{'a': 0.0078527132767012304, 'lan': 0.00011397565932540869, 'e': 2.1797880078135241e-05, 'i': 8.6080888604753787e-05, 'top': 0.24862051383178221, 'aop': 0.0015573146902426677}

The get_elements() method returns two dictionaries, one for the orbital elements and one for their corresponding errors. So this object has a semi-major axis of 111.2246 AU, with an uncertainty of 0.0078 AU. This is a very well measured object, due to its recovery in SDSS observations from over a decade ago. The orbital element "top" is the "time of perihelion" and is given as a Julian date.

### Predicting the position:

Where will this object be during the next DES season?

In [19]:
date_start = ephem.date('2016/08/20')
pos_pred = o.predict_pos(date_start)

In [20]:
print pos_pred

{'dec': -0.07472841993306581, 'opp': 109.40747982653748, 'elong': 108.26099880688548, 'err': {'a': 0.1202744604652362, 'PA': 93.542246880712398, 'b': 0.039854163035020249}, 'ra': 0.7305863501167895}


This returns a dictionary containing the predicted RA and DEC, as well as the opposition angle and solar elongation. The dictionary element 'err' is itself a dictionary with three elements: the major and minor axes of the error ellipse, and the position angle, expressed as degrees from north.

Units can be source of confusion.
  * Errors on predicted positions are in arc-seconds.
  * Positions themselve (RA, DEC) are in <b>radians</b>. To convert to something more familiar, you can use pyephem's conversion routines:

In [21]:
print ephem.hours(pos_pred['ra']), ephem.degrees(pos_pred['dec'])
print ephem.degrees(pos_pred['ra']), ephem.degrees(pos_pred['dec'])
print np.degrees(pos_pred['ra']), np.degrees(pos_pred['dec'])

2:47:26.28 -4:16:53.8
41:51:34.3 -4:16:53.8
41.8595144316 -4.28162307185


Note from above that the positional uncertainty for this object is less than an arc-second. 

The MPC observation format is inconvenient. We often want to fit observations in a more human-readable format, for example the .csv files generated by the <tt>tnofinder</tt> code. pyOrbfit allows you to pass the observations as lists. For example, the file <tt>example_fake.csv</tt> contains observations of a fake TNO reconstructed by our linker in Y2 data. Let's define a utility function:

In [22]:
def fit_orbit(df_obs):
    df_obs = df_obs.ix[['#' not in row['date'] for ind, row in df_obs.iterrows()]]   # filter comment lines
    nobs = len(df_obs)
    ralist = [ephem.hours(r) for r in df_obs['ra'].values]
    declist = [ephem.degrees(d) for d in df_obs['dec'].values]
    datelist = [ephem.date(d) for d in df_obs['date'].values]
    orbit = Orbit(dates=datelist, ra=ralist, dec=declist, obscode=np.ones(nobs, dtype=int)*807, err=0.15)
    return orbit

Now read in the file and fit:

In [24]:
import pandas as pd
df = pd.read_csv('example_fake.csv')

In [25]:
orb = fit_orbit(df)

In [26]:
elements, errs = orb.get_elements()

In [27]:
print elements

{'a': 40.608486321926165, 'lan': 4.302665290341063, 'e': 0.23178926553311102, 'i': 33.03042800311288, 'top': 2473868.675241164, 'aop': 93.54729027856236}


In [28]:
print errs

{'a': 18.58315714983171, 'lan': 0.14371893656525056, 'e': 0.81558563689616881, 'i': 1.7253904772899273, 'top': 17030.563664356956, 'aop': 19.380731360404635}


In [29]:
pos_pred = orb.predict_pos(date_start)
print ephem.hours(pos_pred['ra']), ephem.degrees(pos_pred['dec']), pos_pred['err']['a']

0:31:39.37 6:43:10.5 1107.7020088


At the start of Y4, this "TNO" would have a large 1107-arcsecond positional uncertainty. This is because the discovery arc length in Y2 was only 3 weeks long. This is typical of objects discovered in the wide survey. Efficiently connecting candidates discovered at one opposition with observations from prior or future oppositions is a problem that needs more work.